### Scraping Yelp Reviews from The Original Soupman Restaurant

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

**Using requests to get webpage information from Soupman's Yelp  
Then I will use BeautifulSoup to remove parts of the webpage, most specifically the Review portion**

In [2]:
response = requests.get('https://www.yelp.com/biz/the-original-soupman-new-york-7')
text = BeautifulSoup(response.text, 'html.parser')

In [3]:

# Find the element containing the number of reviews
num_review = text.find(attrs = {'class' : 'css-foyide'})

# Check if the element is found
#if num_reviews:
    # Extract the text containing the number of reviews
#    num_reviews_text = num_reviews_tag.get_text(strip=True)
print(num_review)

<p class="css-foyide">619 reviews</p>


**I was having trouble keeping this code fluid, as you see I had to completely rewrite the num_review variable to extract the integer**

In [4]:
tex = "<p class='css-foyide'>619 reviews</p>"

# Regular expression pattern to match one or more digits
num_reviews_pattern = r'(\d+) reviews'

# Search for the pattern in the text
num_reviews_match = re.search(num_reviews_pattern, tex)

# Check if a match is found
if num_reviews_match:
    # Extract the number of reviews from the match
    num_reviews = int(num_reviews_match.group(1))
    print("Number of reviews:", num_reviews)
else:
    print("Number of reviews not found")

Number of reviews: 619


**With the total number of reviews now in integers, we can use it to access all the review pages and their review texts**

In [5]:
url_list = []
for i in range(0, num_reviews, 10):
    url_list.append('https://www.yelp.com/biz/the-original-soupman-new-york-7?start='+str(i))
print(url_list[:62])

['https://www.yelp.com/biz/the-original-soupman-new-york-7?start=0', 'https://www.yelp.com/biz/the-original-soupman-new-york-7?start=10', 'https://www.yelp.com/biz/the-original-soupman-new-york-7?start=20', 'https://www.yelp.com/biz/the-original-soupman-new-york-7?start=30', 'https://www.yelp.com/biz/the-original-soupman-new-york-7?start=40', 'https://www.yelp.com/biz/the-original-soupman-new-york-7?start=50', 'https://www.yelp.com/biz/the-original-soupman-new-york-7?start=60', 'https://www.yelp.com/biz/the-original-soupman-new-york-7?start=70', 'https://www.yelp.com/biz/the-original-soupman-new-york-7?start=80', 'https://www.yelp.com/biz/the-original-soupman-new-york-7?start=90', 'https://www.yelp.com/biz/the-original-soupman-new-york-7?start=100', 'https://www.yelp.com/biz/the-original-soupman-new-york-7?start=110', 'https://www.yelp.com/biz/the-original-soupman-new-york-7?start=120', 'https://www.yelp.com/biz/the-original-soupman-new-york-7?start=130', 'https://www.yelp.com/biz/the-

**Now I will try to remove the attributes of a yelp review, I will start with one and apply it to the rest once I find that it works**

In [6]:
reviews = text.find_all('div', attrs = {'class' : 'css-1qn0b6x'})
print(len(reviews))

606


In [7]:
review = reviews[0]
#Will try to get username to see if actually grabbed the reviews
username = review.find('span', attrs = {'class': 'fs-block css-ux5mu6'})
print(username)

None


In [8]:
for EachPart in text.select('span[class*="raw__"]'):
     print(EachPart.get_text())
     print("*********")

259A W 55th St
*********
New York, NY 10019
*********
1.6 miles away from The Original Soupman
*********
2.6 miles away from The Original Soupman
*********
Start your review of The Original Soupman
*********
Iconic Soup Kitchen from Seinfeld episode, no soup for you! Had delicious chicken bone broth soup! Great selection of soups. Tip, nice little bakery across the street if you want a brioche roll with your soup!
*********
"No soup for you!"  - Totally stumbled across Soupman on our way to Central Park.  I wasn't even thinking about the place and then we turned the corner and there it was.  It was like I was instantaneously inserted into a Seinfeld episode.  Of course I had to get something.Went with the lobster roll.  The bread was warm and delicious.  The sandwich was a bit on the small side, but for Manhattan and the original Seinfeld Soupman $17 bucks wasn't outrageous.  We also got a chicken dumpling soup of course.  The only thing missing was the no soup for you scream.There is 

In [9]:
def get_reviews_from_urls(url_list):
    all_reviews = []
    for url in url_list:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            reviews = soup.select('span[class*="raw__"]')
            for review in reviews:
                review_text = review.get_text().strip()  # Remove leading/trailing whitespace
                # Filter out reviews with sufficient length
                if len(review_text) >= 42:  # Adjust the threshold as needed
                    all_reviews.append(review_text)
        else:
            print(f"Failed to retrieve data from {url}. Status code: {response.status_code}")
    return all_reviews

In [10]:
reviews = get_reviews_from_urls(url_list)
for review in reviews:
    print(review)
    print("*********")

Iconic Soup Kitchen from Seinfeld episode, no soup for you! Had delicious chicken bone broth soup! Great selection of soups. Tip, nice little bakery across the street if you want a brioche roll with your soup!
*********
"No soup for you!"  - Totally stumbled across Soupman on our way to Central Park.  I wasn't even thinking about the place and then we turned the corner and there it was.  It was like I was instantaneously inserted into a Seinfeld episode.  Of course I had to get something.Went with the lobster roll.  The bread was warm and delicious.  The sandwich was a bit on the small side, but for Manhattan and the original Seinfeld Soupman $17 bucks wasn't outrageous.  We also got a chicken dumpling soup of course.  The only thing missing was the no soup for you scream.There is no indoor dining.  Literally take and go.  So my first time in New York and I'm loving I found Soupman.
*********
Being a Seinfeld fan, I've always wanted to try soup from this establishment. Working in the c

In [11]:
# Iterate over each review in the reviews list and print its length
for review in reviews:
    print(len(review))

209
675
926
423
669
107
411
688
1006
248
484
97
346
342
423
629
440
584
1178
875
318
281
567
277
291
282
172
1776
333
356
353
496
508
105
1714
373
373
1016
717
260
487
397
266
144
398
810
149
323
495
204
169
992
205
99
330
247
199
523
413
122
159
187
678
252
474
880
166
621
115
245
245
700
418
815
190
225
85
502
99
190
155
528
127
470
808
486
290
108
757
536
472
830
348
772
280
114
463
444
259
132
831
643
348
832
84
90
146
333
107
709
1182
529
175
613
257
282
105
603
122
1430
1190
246
511
474
197
214
848
199
95
86
183
1215
707
726
410
610
233
260
317
332
149
188
424
412
106
129
304
725
160
157
163
462
465
390
439
251
905
566
1190
857
714
450
486
351
1218
448
291
638
208
584
86
772
445
806
668
250
290
384
905
415
603
518
463
91
377
121
420
125
1408
112
349
259
755
596
278
363
219
350
1090
459
569
123
840
538
205
533
147
286
283
820
1947
304
440
749
342
237
256
199
383
470
361
156
191
1292
411
304
639
925
178
178
472
668
386
273
437
460
1051
154
568
301
167
345
357
218
182
1155
145
248
7

**Now that I was able to get nearly all the reviews (604 should be good enough) I will turn the data into a dataframe, as well as a csv so that I can begin sentiment analysis**

In [12]:
reviews_df = pd.DataFrame(reviews, columns =['review'])

In [14]:
reviews_df['review'] = reviews_df['review'].str.strip()

**Stripping the records for leading and trailing whitespace just in case**

In [15]:
reviews_df.to_csv('soupman_reviews.csv',index = False)